In [2]:
# train_los_models.py
# Train multiple LOS models and save them as sklearn pipelines.

import os
from math import sqrt
from pathlib import Path

import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    HistGradientBoostingRegressor,
)
from sklearn.neural_network import MLPRegressor

# -----------------------------
# Config
# -----------------------------
RANDOM_STATE = 100
TRAIN_PCT = 0.70
file_path = Path("..") / "Data" / "LengthOfStay.csv"  # adjust if needed

TARGET = "lengthofstay"
ID_COL = "eid"
DATE_COLS = ["vdate", "discharged"]
DROP_FROM_FEATURES = ["eid", "vdate", "discharged", "facid"]

CONTINUOUS_TO_STANDARDIZE = [
    "hematocrit",
    "neutrophils",
    "sodium",
    "glucose",
    "bloodureanitro",
    "creatinine",
    "bmi",
    "pulse",
    "respiration",
]

ISSUE_INDICATORS = [
    "hemo",
    "dialysisrenalendstage",
    "asthma",
    "irondef",
    "pneum",
    "substancedependence",
    "psychologicaldisordermajor",
    "depress",
    "psychother",
    "fibrosisandother",
    "malnutrition",
]

PROCEDURE_COL_CANDIDATES = [
    "procedure",
    "primaryprocedure",
    "surgery",
    "aprdrgdescription",
]

# -----------------------------
# Utility
# -----------------------------
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print(f"[{model_name}] MAE={mae:.4f} | RMSE={rmse:.4f} | R^2={r2:.4f}")
    return {"model": model_name, "MAE": mae, "RMSE": rmse, "R2": r2}


# -----------------------------
# Step 1: Load, type cast, clean NA
# -----------------------------
df = pd.read_csv(file_path)

# Parse dates
for c in DATE_COLS:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors="coerce")

# Ensure indicator columns numeric
present_issue_cols = [c for c in ISSUE_INDICATORS if c in df.columns]
for c in present_issue_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

protected = {ID_COL, TARGET, *DATE_COLS}
cols_to_consider = [c for c in df.columns if c not in protected]

num_cols_all = df[cols_to_consider].select_dtypes(include=["number"]).columns.tolist()
cat_cols_all = df[cols_to_consider].select_dtypes(
    include=["object", "category"]
).columns.tolist()

if num_cols_all:
    means = df[num_cols_all].mean(numeric_only=True)
    df[num_cols_all] = df[num_cols_all].fillna(means)

for c in cat_cols_all:
    if df[c].isna().any():
        mode_val = df[c].mode(dropna=True)
        df[c] = df[c].fillna(mode_val.iloc[0] if not mode_val.empty else "UNKNOWN")

df[TARGET] = pd.to_numeric(df[TARGET], errors="coerce")

print("Step 1 complete: data cleaned.")

# -----------------------------
# Step 2: Feature engineering
# -----------------------------
present_cont = [c for c in CONTINUOUS_TO_STANDARDIZE if c in df.columns]
for c in present_cont:
    mean = df[c].mean()
    std = df[c].std(ddof=0)
    df[c] = (df[c] - mean) / (std if std else 1.0)

if present_issue_cols:
    df["number_of_issues"] = (
        df[present_issue_cols]
        .apply(pd.to_numeric, errors="coerce")
        .fillna(0)
        .sum(axis=1)
        .astype(int)
        .astype(str)
    )
else:
    df["number_of_issues"] = "0"

print("Step 2 complete: standardized labs and created number_of_issues.")

# -----------------------------
# Procedure stats (for reference / later use)
# -----------------------------
PROC_COL = next((c for c in PROCEDURE_COL_CANDIDATES if c in df.columns), None)
if PROC_COL:
    proc_stats = df.groupby(PROC_COL)[TARGET].agg(["mean", "std", "count"])
    print(f"Using procedure column: {PROC_COL}")
else:
    proc_stats = None
    print("No procedure column found among candidates.")

# -----------------------------
# Prepare features/target and split
# -----------------------------
feature_cols = [c for c in df.columns if c not in set(DROP_FROM_FEATURES + [TARGET])]
X = df[feature_cols].copy()
y = df[TARGET].copy()

cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object", "category"]).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=TRAIN_PCT, random_state=RANDOM_STATE
)
print(f"Split: train={len(X_train)}, test={len(X_test)}")

# Preprocessor
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", ohe, cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# -----------------------------
# Step 3: Train many models
# -----------------------------
results = []
oob_min_samples_split = int(max(2, round(sqrt(len(X_train)))))

models = {
    # Linear family (per-feature contributions via coef_)
    "LIN_OLS": LinearRegression(),
    "LIN_RIDGE": Ridge(alpha=1.0),
    "LIN_LASSO": Lasso(alpha=0.01, max_iter=5000),
    "LIN_ELASTIC": ElasticNet(alpha=0.01, l1_ratio=0.5, max_iter=5000),

    # Tree-based models (feature_importances_)
    "RF_100": RandomForestRegressor(
        n_estimators=100,
        random_state=5,
        oob_score=True,
        bootstrap=True,
        n_jobs=-1,
        min_samples_split=oob_min_samples_split,
    ),
    "GBT_40": GradientBoostingRegressor(
        n_estimators=40,
        learning_rate=0.3,
        random_state=9,
    ),
    "HGB_100": HistGradientBoostingRegressor(
        max_iter=100,
        learning_rate=0.1,
        random_state=RANDOM_STATE,
    ),

    # Neural net / black-box
    "NN_64x2": MLPRegressor(
        hidden_layer_sizes=(64, 64),
        activation="relu",
        solver="adam",
        max_iter=500,
        random_state=17,
    ),
}

fitted_pipelines = {}
for name, model in models.items():
    pipe = Pipeline(steps=[("prep", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    fitted_pipelines[name] = pipe

    res = evaluate_model(y_test, y_pred, name)
    if name.startswith("RF") and hasattr(pipe.named_steps["model"], "oob_score_"):
        print(f"[{name}] OOB R^2={pipe.named_steps['model'].oob_score_:.4f}")
    results.append(res)

metrics_df = pd.DataFrame(results).sort_values("RMSE")
print("\nSummary metrics (sorted by RMSE):")
print(metrics_df)

# -----------------------------
# Save models
# -----------------------------
out_dir = Path("./models_sklearn_los")
out_dir.mkdir(parents=True, exist_ok=True)
for name, pipe in fitted_pipelines.items():
    joblib.dump(pipe, out_dir / f"{name}_model.joblib")
print(f"Saved {len(fitted_pipelines)} models to {out_dir.resolve()}")

# -----------------------------
# Optional: build predictions tables for a few models
# -----------------------------
def build_predictions_table(model_key: str):
    assert model_key in fitted_pipelines, f"Unknown model_key {model_key}"
    pipe = fitted_pipelines[model_key]

    pred = pipe.predict(X_test)
    pred_round = np.rint(pred).astype(int)

    include_cols = [
        "eid",
        "vdate",
        "rcount",
        "gender",
        "discharged",
        "facid",
        "lengthofstay",
    ]
    if PROC_COL:
        include_cols.append(PROC_COL)

    present_cols = [c for c in include_cols if c in df.columns]
    base = df.loc[X_test.index, present_cols].copy()

    if "vdate" in base.columns:
        base["vdate"] = pd.to_datetime(base["vdate"], errors="coerce")

    base["lengthofstay_Pred"] = pred
    base["lengthofstay_Pred_Rounded"] = pred_round
    if "vdate" in base.columns:
        base["discharged_Pred"] = base["vdate"] + pd.to_timedelta(
            base["lengthofstay_Pred_Rounded"], unit="D"
        )
    else:
        base["discharged_Pred"] = pd.NaT

    if PROC_COL and proc_stats is not None and PROC_COL in base.columns:
        base["procedure_AvgLoS"] = base[PROC_COL].map(proc_stats["mean"])
        base["procedure_LoS_Std"] = base[PROC_COL].map(proc_stats["std"])

    order = [
        "eid",
        "vdate",
        "lengthofstay",
        "lengthofstay_Pred",
        "lengthofstay_Pred_Rounded",
        "discharged_Pred",
    ]
    if PROC_COL and PROC_COL in base.columns:
        order.append(PROC_COL)
        order += ["procedure_AvgLoS", "procedure_LoS_Std"]
    order += [c for c in present_cols if c not in order]
    cols_final = [c for c in order if c in base.columns]
    return base[cols_final]


out_dir_preds = Path("./predictions_sklearn_los")
out_dir_preds.mkdir(parents=True, exist_ok=True)
for key in ["GBT_40", "RF_100", "LIN_OLS"]:
    if key in fitted_pipelines:
        tbl = build_predictions_table(key)
        tbl.to_csv(out_dir_preds / f"LoS_Predictions_{key}.csv", index=False)

print(f"Prediction tables written to {out_dir_preds.resolve()}")


Step 1 complete: data cleaned.
Step 2 complete: standardized labs and created number_of_issues.
No procedure column found among candidates.
Split: train=70000, test=30000
[LIN_OLS] MAE=0.8449 | RMSE=1.1047 | R^2=0.7820
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
[LIN_RIDGE] MAE=0.8449 | RMSE=1.1047 | R^2=0.7820
[LIN_LASSO] MAE=0.8516 | RMSE=1.1205 | R^2=0.7758
[LIN_ELASTIC] MAE=0.8479 | RMSE=1.1180 | R^2=0.7768
[RF_100] MAE=0.4924 | RMSE=0.7858 | R^2=0.8897
[RF_100] OOB R^2=0.8917
[GBT_40] MAE=0.3675 | RMSE=0.4854 | R^2=0.9579
[HGB_100] MAE=0.3110 | RMSE=0.4310 | R^2=0.9668
[NN_64x2] MAE=0.3253 | RMSE=0.4753 | R^2=0.9597

Summary metrics (sorted by RMSE):
         model       MAE      RMSE        R2
6      HGB_100  0.311015  0.430963  0.966829
7      NN_64x2  0.325252  0.475304  0.95